In [1]:
%cd /content/drive/MyDrive/
# raw_data is imported from global config
root_dir='/content/drive/MyDrive/'  # ChangeThis
result_dir=root_dir+'grasp/TSception/result_test_dropout/'

/content/drive/MyDrive


In [2]:
import sys, importlib
#importlib.reload(sys.modules['grasp.config'])
from grasp.config import data_dir
# orveride the data_dir in config file
#data_dir='/content/drive/MyDrive/data/' # googleDrive

In [3]:
data_dir

'/content/drive/MyDrive/data/'

In [4]:
! pip install mne==0.19.2;
! pip install torch;

     |████████████████████████████████| 6.4MB 9.5MB/s 


In [5]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

from grasp.TSception.utils import regulization
from grasp.utils import SEEGDataset,SEEGDataset1
from grasp.TSception.Models import TSception

# load the data: regression to target force derivative
from grasp.utils import rawData2
from grasp.config import activeChannels, root_dir


In [6]:
# reload function
import sys, importlib
importlib.reload(sys.modules['grasp.TSception.Models'])
importlib.reload(sys.modules['grasp.utils'])

from grasp.utils import SEEGDataset,SEEGDataset1
from grasp.TSception.Models import TSception,TSception2
from grasp.utils import rawData2

In [7]:
# import from another folder
import sys
sys.path.insert(1, '/content/drive/MyDrive/examples')
from IMV_LSTM.networks import IMVTensorLSTM

In [8]:
data_dir

'/content/drive/MyDrive/data/'

In [9]:
import inspect as i
import sys
#sys.stdout.write(i.getsource(rawData2));

In [10]:
enable_cuda = torch.cuda.is_available()
print('GPU computing: ', enable_cuda)

GPU computing:  True


In [11]:
%%capture
# suppress the output
sampling_rate=1000
#traindata, valdata, testdata = rawData2('raw',activeChannels,move2=False)  # (chns, 15000/15001, 118) (channels, time, trials)
traindata, valdata, testdata = rawData2('band','all',move2=True)
traindata = traindata.transpose(2, 0, 1)  # (118, 20, 15000) (trials,channels,  time)
valdata = valdata.transpose(2, 0, 1) # (8, 20, 15000)
testdata = testdata.transpose(2, 0, 1)  # (8, 20, 15000)
trainx_ds, trainy_ds = traindata[:, :-2, :], traindata[:, -2, :] #-2 is real force, -1 is target
valx_ds, valy_ds = valdata[:, :-2, :], valdata[:, -2, :]
testx_ds, testy_ds = testdata[:, :-2, :], testdata[:, -2, :]


In [12]:
chnNum=trainx_ds.shape[1]
learning_rate=0.002
epochs=100
step=50 #ms
T=1000 #ms
totalLen=trainx_ds.shape[2] #ms
batch_size=int((totalLen-T)/step) # 280
num_T = 3 # (6 conv2d layers) * ( 3 kernel each layer)
num_S = 3
hidden_size=222
dropout=0.2
Lambda = 1e-6

dataset_train = SEEGDataset1(trainx_ds, trainy_ds,T,step)
dataset_val = SEEGDataset1(valx_ds, valy_ds,T,step)
dataset_test = SEEGDataset1(testx_ds, testy_ds,T,step)

In [13]:
len(dataset_train)

118

In [14]:
#trainy.shape
#plt.plot(trainy_ds[0,:])

In [15]:
train_loader = DataLoader(dataset=dataset_train, batch_size=1, shuffle=True, pin_memory=False)
val_loader = DataLoader(dataset=dataset_val, batch_size=1, pin_memory=False)
test_loader = DataLoader(dataset=dataset_test, batch_size=1, pin_memory=False)

In [16]:
# __init__(self,input_size, sampling_rate, num_T, num_S, hiden, dropout_rate)
net = TSception2(chnNum,sampling_rate, num_T, num_S,batch_size).float()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
if(enable_cuda):
	net.cuda()
 
checkpoint = torch.load(result_dir+'checkpoint195.pth')
net.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [17]:
debugg=False
#debugg=True
for epoch in range(196,300):
    print("------ epoch " + str(epoch) + " -----")
    net.train()

    loss_epoch=0
    #trial=0
    for trial, (trainx, trainy) in enumerate(train_loader): # ([1, 15000, 19]), ([1, 15000])
        #trainy[0,-1]+=0.05
        if debugg==True: # just test one trial
            if trial == 1:
                break
                pass
        optimizer.zero_grad()
        
        if (enable_cuda):
            x= trainx.float().cuda()
            target = trainy.float().cuda()
        y_pred = net(x)
        #target = torch.from_numpy(target)

        # regularization
        loss1 = criterion(y_pred, target.float())
        loss2 = regulization(net, Lambda)
        #loss3 = y_pred.cpu().detach().numpy()
        #loss3 = np.std(np.diff(loss3.reshape(-1)))
        loss=loss1+loss2 #+loss3*0.001
        loss.backward()
        optimizer.step()

        ls=loss1.item()
        loss_epoch+=ls
        with open(result_dir+ "trainlose.txt", "a") as f:
            f.write(str(loss1) + "\n")
    print(""+str(epoch)+" loss:"+str(loss_epoch/(trial+1))+".")
    if epoch % 1 ==0:
        net.eval()
        print("Validating...")
        with torch.no_grad():
            vpredAll = []
            vtargetAll = []
            for trial, (vx, vtarget) in enumerate(val_loader):  # ([1, 15000, 19]), ([1, 15000])                
                if (enable_cuda):
                    vx= vx.float().cuda()
                    vtarget = vtarget.float().cuda()
                y_pred = net(vx)
                loss3 = criterion(y_pred.squeeze(), vtarget.squeeze())
                with open(result_dir+"testlose.txt", "a") as f:
                    f.write(str(loss3) + "\n")

                y_pred=y_pred.squeeze().cpu().detach().numpy()
                vtarget=vtarget.squeeze().cpu().numpy()
                vpredAll.append(y_pred)
                vtargetAll.append(vtarget)


        vpredAll = np.concatenate(vpredAll,axis=0)
        vtargetAll = np.concatenate(vtargetAll, axis=0)

        fig, ax = plt.subplots(figsize=(6, 3))
        plt.ion()
        ax.clear()
        ax.plot(vtargetAll, label="True", linewidth=1)
        ax.plot(vpredAll, label='Predicted - Test', linewidth=1)
        ax.legend(loc='upper left')
        figname = result_dir+'prediction' + str(epoch) + '.png'
        fig.savefig(figname)
        plt.close(fig)
    if epoch % 5==0:
        state = {
            'state_dict': net.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        savepath = result_dir+'checkpoint'+str(epoch)+'.pth'
        torch.save(state, savepath)


------ epoch 196 -----
196 loss:0.04963546498852261.
Validating...
------ epoch 197 -----
197 loss:0.04723650940015155.
Validating...
------ epoch 198 -----
198 loss:0.04688021172059037.
Validating...
------ epoch 199 -----
199 loss:0.04671997752166906.
Validating...
------ epoch 200 -----
200 loss:0.04777985209030872.
Validating...
------ epoch 201 -----
201 loss:0.04648004002648137.
Validating...
------ epoch 202 -----
202 loss:0.04702675697330575.
Validating...
------ epoch 203 -----
203 loss:0.0463242298702441.
Validating...
------ epoch 204 -----
204 loss:0.044910610274616944.
Validating...
------ epoch 205 -----
205 loss:0.04367270982445423.
Validating...
------ epoch 206 -----
206 loss:0.046664377208799124.
Validating...
------ epoch 207 -----
207 loss:0.04626473114368016.
Validating...
------ epoch 208 -----
208 loss:0.04822488392750591.
Validating...
------ epoch 209 -----
209 loss:0.044103731874818516.
Validating...
------ epoch 210 -----
210 loss:0.04735364326891505.
Validat